## Gene tree estimation error in sliding windows

What size window is too big such that concatenation washes away the differences among genealogies for MSC-based analyses (i.e., ASTRAL, SNAQ).

In [24]:
import toytree
import ipcoal
import numpy as np
import ipyrad.analysis as ipa

### Set up a phylogenetic model

In [9]:
tree = toytree.rtree.unittree(ntips=12, treeheight=12e6, seed=123)

In [10]:
tree.draw(ts='p');

<svg class="toyplot-canvas-Canvas" height="265.0px" id="t1f4bc3de69f745a3b7d8d4aad1b02e49" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 265.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 r10 r11 0 6000000 12000000

### Simulate a chromosome

In [11]:
model = ipcoal.Model(
    tree=tree,
    Ne=1e6,
    nsamples=2,
    mut=1e-08,
    recomb=1e-09,
)

In [12]:
model.sim_loci(nloci=1, nsites=1e5)

### Add missing data as spacers between loci and allele dropout

In [13]:
# assumed space between RAD tags
SPACER = 5000
CUTLEN = 5

# iterate over each RAD tag
for i in range(0, model.seqs.shape[2], SPACER):
    
    # mask. [0-300=DATA][300-5300=SPACER]
    model.seqs[:, :, i+300: i+SPACER] = 9
    
    # allele dropout
    cseqs = model.seqs[:, :, i:i+CUTLEN]
    aseqs = model.ancestral_seq[0, i:i+CUTLEN]
    mask = np.any(cseqs != aseqs, axis=2)[0]
    model.seqs[:, mask, i:i+300] = 9

In [14]:
# check that data looks right
model.draw_seqview(0, 250, 350, height=800);

<svg class="toyplot-canvas-Canvas" height="800.0px" id="t4b07135284c043188a77a6da1496d552" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 1980.0 800.0" width="1980.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r0-0 r0-1 r1-0 r1-1 r10-0 r10-1 r11-0 r11-1 r2-0 r2-1 r3-0 <rect height="25.229166666666686" style="fill:rgb(50.2%,50.2%,50.2%);fill-opacity:1.0;opacity:0.5;stroke:none" width="13.839999999999918" x

### Write data to SEQS HDF5 format

In [15]:
model.write_loci_to_hdf5(name="test", outdir="/tmp", diploid=True)

wrote 1 loci to /tmp/test.seqs.hdf5


### Reformat all genealogies for comparisons with inferred gene trees
The true gene trees will not distinguish among haplotypes, so we will drop one haplotype from each tip, and we will also multiply branch lengths by the mutation rate so that edge lengths are in units of mutations.

In [17]:
def convert_genealogy_to_gene_tree(gtree, mu=1e-8):
    # multiply by mutation rate
    gtree = gtree.set_node_values(
        feature="dist", 
        values={i: j.dist * mu for (i, j) in gtree.idx_dict.items()},
    )
    
    # drop the -1 haplotype from each
    gtree = gtree.drop_tips([i for i in gtree.get_tip_labels() if "-1" in i])
    
    # drop -0 from names of remaining samples
    gtree = gtree.set_node_values(
        feature="name", 
        values={i: j.name[:-2] for (i, j) in gtree.idx_dict.items()},
    )
    return gtree

In [18]:
# convert genealogies to be gene-tree-like
model.df.genealogy = [
    convert_genealogy_to_gene_tree(toytree.tree(i)).write() 
    for i in model.df.genealogy
]

### Save record of the TRUE genealogy at each position

In [16]:
model.df.to_csv("/tmp/test.csv")

### Visualize tree variation

In [22]:
# show the first few trees
toytree.mtree(model.df.genealogy[:10]).draw(2, 4, height=500);

<svg class="toyplot-canvas-Canvas" height="500.0px" id="tc1bf6686d32a40fcb6d9b6b34c251f5b" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 560.0 500.0" width="560.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r6 r9 r7 r8 r10 r11 r5 r4 r2 r3 r0 r1 r6 r2 r3 r0 r1 r9 r7 r8 r10 r11 r4 r5 r6 r2 r3 r0 r1 r9 r7 r8 r10 r11 r4 r5 r6 r2 r3 r0 r1 r7 r8 r9 r10 r11 r4 r5 r6 r2 r3 r0 r1 r7 r8 r9 r10 r11 r4 r5 r6 r2 r3 r0 r1 r7 r8 r9 r10 r11 r4 r5 r6 r2 r3 r0 r1 r7 r8 r9 r10 r11 r4 r5 r6 r2 r3 r0 r1 r7 r8 r9 r10 r11 r4 r5

### Infer gene trees in sliding windows along the chromosome

In [26]:
# raxml inference in sliding windows
ts = ipa.treeslider("/tmp/test.seqs.hdf5", window_size=5e4, slide_size=5e4)
ts.run(auto=True, force=True)

building database: nwindows=1; minsnps=1
[####################] 100% 0:00:20 | inferring trees 
tree_table written to /home/deren/Documents/toytree/sandbox/analysis-treeslider/test.tree_table.csv


In [28]:
# inferred tree is unrooted
toytree.tree(ts.tree_table.tree[0]).draw();

<svg class="toyplot-canvas-Canvas" height="275.0px" id="tdd6185a5b53446fdba4dd3fa8a6185f8" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 265.0 275.0" width="265.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r7 r8 r9 r10 r11 r5 r6 r4 r3 r1 r0 r2

### Infer a species tree from inferred gene trees

In [30]:
# infer sptree from inferred gene trees from windows
ast = ipa.astral(ts.tree_table)
ast.run()
toytree.tree(ast.tree).draw();

[astral.5.7.3.jar]
inferred tree written to (/home/deren/Documents/toytree/sandbox/analysis-astral/test.tre)


<svg class="toyplot-canvas-Canvas" height="275.0px" id="t5f3693339284423486616b7098ee621d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 265.0 275.0" width="265.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r7 r8 r9 r10 r11 r5 r6 r4 r3 r0 r1 r2

### Infer a species tree from TRUE gene trees

In [32]:
# sample one tree every 5000bp
gtrees = []

# select gtree every SPACER LEN bp (THIS IS THE SIZE OF WINDOWS)
for point in range(0, model.df.end.max(), 5000):
    
    # get first tree with start > point
    gtree = model.df.loc[model.df.start >= point, "genealogy"].iloc[0]
    gtrees.append(gtree)

In [33]:
import ipyrad.analysis as ipa
ast = ipa.astral(gtrees)
ast.run()
ast.tree.draw();

[astral.5.7.3.jar]
inferred tree written to (/home/deren/Documents/toytree/sandbox/analysis-astral/test.tre)


<svg class="toyplot-canvas-Canvas" height="275.0px" id="tcbd86ff1fed44a81b61ff89dba5d53b8" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 265.0 275.0" width="265.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> r9 r8 r7 r10 r11 r6 r5 r4 r3 r2 r0 r1

### Measure RF distance between trees
The normalized RF distance. Larger value means trees are more different.

In [142]:
# get two toytrees to compare
tree1 = toytree.tree(model.df.genealogy[0])
tree2 = toytree.tree(model.df.genealogy[100])

In [143]:
# calculate normalized RF distance
rf, rfmax, _, _, _, _, _ = tree1.treenode.robinson_foulds(tree2.treenode)
print(rf, rfmax, rf / rfmax)

12 92 0.13043478260869565


In [159]:
# unresolved tree example RF calc
unresolved = tree1.collapse_nodes(min_dist=5e6)
rf, rfmax, _, _, _, _, _ = unresolved.treenode.robinson_foulds(tree2.treenode, unrooted_trees=True)
print(rf, rfmax, rf / rfmax)

12 78 0.15384615384615385


### Visualize gene tree error
Some kind of sliding plot ...

In [ ]:

chrom   ----------------------------------------------------------------
windows --------- ----------  ------------
RAD loc     -       -    -      - -     -  

gt erro ---        ---       ---      ---


# separate figure
windowsize x spptree error (astral)